In [1]:
# Import all the required libraries

import os;
import numpy as np;
import pandas as pd;

Importing the input data from an S3 bucket

In [2]:
s3_bkt = "s3://aws-poc-shweta"
s3_json_in = s3_bkt + "/input_data/SCalcET8_out.json"
df = pd.read_json(s3_json_in,'index')

In [3]:
df.head(3)

,cendoc,chaptername,narrativeid,plaintext,sectionname,titlename
SCalcET8 1.1,"<cl:sect2 xmlns:cl=""http://xml.cengage-learnin...",Functions and Models,SCalcET8 1.1,Four Ways to Represent a Function 1.1 Functi...,Four Ways to Represent a Function,Calculus: Early Transcendentals
SCalcET8 1.2,"<cl:sect2 xmlns:cl=""http://xml.cengage-learnin...",Functions and Models,SCalcET8 1.2,Mathematical Models: A Catalog of Essential F...,Mathematical Models: A Catalog of Essential Fu...,Calculus: Early Transcendentals
SCalcET8 1.3,"<cl:sect2 xmlns:cl=""http://xml.cengage-learnin...",Functions and Models,SCalcET8 1.3,New Functions from Old Functions 1.3 In this...,New Functions from Old Functions,Calculus: Early Transcendentals


Specifying the output location

In [4]:
# output CSV with just id and text columns - used as input to comprehend topic model

#specify the location
s3_csv_out = s3_bkt + "/output_data/narrative.csv"
df.to_csv('/tmp/narrative.csv', columns = ['narrativeid','chaptername','sectionname','plaintext'], index=False, header=False) #temp file
!aws s3 cp '/tmp/narrative.csv' '{s3_csv_out}' #copy temp file to output location

upload: ../../../../tmp/narrative.csv to s3://aws-poc-shweta/output_data/narrative.csv


## 2. Topic Modeling using Comprehend

In [5]:
import time
from time import gmtime, strftime
import boto3

In [6]:
#creating the client
client = boto3.client('comprehend')

In [7]:
client

In [27]:
num_topics = 20

In [33]:
s3_comprehend_out = s3_bkt + "/output_data/comprehend_out/" #where the results will be written

#The role used , use this from IAM console 
#(roles > click on role created when the notebook instance was created ; summary > role ARN)
comprehend_role = 'arn:aws:iam::684066638289:role/service-role/AmazonSageMaker-ExecutionRole-20190604T130317'

#Might need to attach the appropriate policy to the Sagemaker role created in order to run comprehend jobs


#Starting the Comprehend Job. 
response = client.start_topics_detection_job(
    InputDataConfig={
        'S3Uri': s3_csv_out,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': s3_comprehend_out
    },
    DataAccessRoleArn=comprehend_role,
    JobName='narrative_topic_model',
    NumberOfTopics=num_topics
)

In [34]:
jobid = response['JobId']
status = response['JobStatus']

In [35]:
while status in ['SUBMITTED', 'IN_PROGRESS']:
    response = client.describe_topics_detection_job(
        JobId=jobid
    )
    status = response['TopicsDetectionJobProperties']['JobStatus']
    print(strftime("%H:%M:%S", gmtime()) + ": " + status)
    time.sleep(120)

18:19:07: FAILED
